<a href="https://colab.research.google.com/github/takaaki-yayoi/report_analysis_delivery/blob/main/%E3%83%AC%E3%83%9D%E3%83%BC%E3%83%88%E4%BD%9C%E6%88%90%E5%8F%8A%E3%81%B3%E9%85%8D%E5%B8%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 必要なライブラリのインストール

In [33]:
# OpenAI最新版をインストール
!pip install -q openai>=1.0.0 python-docx python-pptx openpyxl
!pip install -q matplotlib seaborn japanize-matplotlib

# Google Driveのマウント（レポート保存用）
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## ライブラリのインポートと初期設定

In [34]:
import os
import json
import pandas as pd
import numpy as np
from datetime import datetime
from typing import Dict, List, Any, Optional
import warnings
warnings.filterwarnings('ignore')

# グラフ作成用
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib  # 日本語表示対応

# Word/PowerPoint作成用
from docx import Document
from docx.shared import Inches, Pt, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
from pptx import Presentation
from pptx.util import Inches, Pt

# Google Colab用設定
from google.colab import files, userdata
from IPython.display import display, HTML, Image
import io
import base64

# OpenAI新バージョンのインポート
from openai import OpenAI

print("✅ ライブラリのインポート完了")

✅ ライブラリのインポート完了


## OpenAI APIキーの設定

In [35]:
# OpenAI APIキーの設定
# 方法1: Colabのシークレット機能を使用（推奨）
try:
    api_key = userdata.get('openai_api_key')
    client = OpenAI(api_key=api_key)
    print("✅ OpenAI APIキーが設定されました（シークレット）")
except:
    # 方法2: 直接入力
    import getpass
    api_key = getpass.getpass('OpenAI APIキーを入力してください: ')
    client = OpenAI(api_key=api_key)
    print("✅ OpenAI APIキーが設定されました")

# グローバル変数として設定
OPENAI_CLIENT = client

✅ OpenAI APIキーが設定されました（シークレット）


## 設定値の定義

In [36]:
# システム設定
GPT_MODEL = "gpt-4o-mini"
MAX_TOKENS = 1500
REPORT_FORMAT = "both"  # "word", "ppt", または "both"
SAVE_TO_DRIVE = True
DOWNLOAD_REPORTS = False

# 会社方針（カスタマイズ可能）
COMPANY_POLICY = """
1. 顧客満足度の向上
2. イノベーションの推進
3. 働きやすい職場環境の実現
4. 持続可能な成長
5. チームワークの強化
"""

print("✅ 設定完了")
print(f"モデル: {GPT_MODEL}")
print(f"レポート形式: {REPORT_FORMAT}")
print(f"Drive保存: {SAVE_TO_DRIVE}")

✅ 設定完了
モデル: gpt-4o-mini
レポート形式: both
Drive保存: True


## サンプルデータの生成

In [37]:
# サンプルデータの生成
np.random.seed(42)

departments = ['営業部', '開発部', '人事部', '経理部', '企画部']
dates = pd.date_range('2024-01-01', periods=90, freq='D')

data_list = []
comments = [
    'チームワークが向上しています。',
    '業務効率化のツールが必要です。',
    'コミュニケーションが改善されました。',
    'リモートワークの環境を整備してほしい。',
    '研修プログラムが役立っています。',
    '残業時間を減らす工夫が必要。',
    '新しいシステムの導入効果が出ています。',
    'もっと権限委譲を進めてほしい。',
    '職場の雰囲気が良くなっています。',
    'キャリアパスを明確にしてほしい。'
]

for dept in departments:
    for i in range(20):
        data_list.append({
            '部門': dept,
            '日付': np.random.choice(dates),
            '従業員ID': f'{dept[:2]}{i+1:03d}',
            '満足度スコア': np.random.randint(60, 95),
            '生産性スコア': np.random.randint(55, 90),
            'チームワークスコア': np.random.randint(50, 95),
            'イノベーションスコア': np.random.randint(45, 85),
            'コメント': np.random.choice(comments)
        })

df = pd.DataFrame(data_list)
df = df.sort_values(['部門', '日付'])

print("📊 生成されたサンプルデータ:")
print(f"データ件数: {len(df)}件")
print(f"部門数: {df['部門'].nunique()}部門")
display(df.head(10))

📊 生成されたサンプルデータ:
データ件数: 100件
部門数: 5部門


,部門,日付,従業員ID,満足度スコア,生産性スコア,チームワークスコア,イノベーションスコア,コメント
49,人事部,2024-01-04,人事010,94,71,93,72,研修プログラムが役立っています。
50,人事部,2024-01-06,人事011,94,78,78,75,コミュニケーションが改善されました。
45,人事部,2024-01-08,人事006,66,57,66,77,コミュニケーションが改善されました。
48,人事部,2024-01-30,人事009,92,82,82,49,コミュニケーションが改善されました。
41,人事部,2024-02-02,人事002,83,65,57,80,残業時間を減らす工夫が必要。
55,人事部,2024-02-02,人事016,68,83,91,70,コミュニケーションが改善されました。
57,人事部,2024-02-05,人事018,79,55,57,60,コミュニケーションが改善されました。
40,人事部,2024-02-17,人事001,75,57,69,68,残業時間を減らす工夫が必要。
56,人事部,2024-02-19,人事017,84,78,62,51,職場の雰囲気が良くなっています。
47,人事部,2024-02-23,人事008,67,81,76,78,研修プログラムが役立っています。


## メインクラスの定義（パート1）

In [38]:
class ColabDepartmentReportAutomation:
    """Google Colab用 部門別レポート自動生成システム"""

    def __init__(self, model="gpt-4", max_tokens=1500, company_policy=""):
        """初期化処理"""
        self.model = model
        self.max_tokens = max_tokens
        self.company_policy = company_policy or COMPANY_POLICY

        # レポート保存用ディレクトリ
        self.base_dir = '/content/drive/MyDrive/DepartmentReports' if SAVE_TO_DRIVE else '/content/reports'
        os.makedirs(self.base_dir, exist_ok=True)
        os.makedirs(f'{self.base_dir}/graphs', exist_ok=True)

    def _calculate_trend(self, series):
        """トレンドの計算"""
        if len(series) < 2:
            return "データ不足"

        x = np.arange(len(series))
        y = series.values
        mask = ~np.isnan(y)

        if mask.sum() < 2:
            return "データ不足"

        x, y = x[mask], y[mask]
        slope = np.polyfit(x, y, 1)[0]

        if slope > 0.1:
            return "📈 上昇傾向"
        elif slope < -0.1:
            return "📉 下降傾向"
        else:
            return "➡️ 横ばい"

    def _create_graphs(self, data, department, score_columns):
        """グラフの作成"""
        graphs = {}
        colors = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D', '#6A994E']

        # 1. レーダーチャート
        if len(score_columns) >= 3:
            fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(projection='polar'))

            angles = np.linspace(0, 2 * np.pi, len(score_columns), endpoint=False).tolist()
            values = [data[col].mean() for col in score_columns]
            values += values[:1]
            angles += angles[:1]

            ax.plot(angles, values, 'o-', linewidth=2, color=colors[0])
            ax.fill(angles, values, alpha=0.25, color=colors[0])
            ax.set_xticks(angles[:-1])
            ax.set_xticklabels(score_columns, size=10)
            ax.set_ylim(0, 100)
            ax.set_title(f'{department} - スコア総合評価', size=14, pad=20)
            ax.grid(True)

            filepath = f'{self.base_dir}/graphs/{department}_radar.png'
            plt.savefig(filepath, dpi=100, bbox_inches='tight', facecolor='white')
            plt.close()
            graphs['radar'] = filepath

        # 2. 時系列グラフ
        if '日付' in data.columns and len(score_columns) > 0:
            fig, ax = plt.subplots(figsize=(12, 6))

            for i, col in enumerate(score_columns[:4]):
                rolling_mean = data.groupby('日付')[col].mean().rolling(window=7, min_periods=1).mean()
                ax.plot(rolling_mean.index, rolling_mean.values,
                       marker='o', label=col, color=colors[i % len(colors)], linewidth=2)

            ax.set_title(f'{department} - スコア推移（7日移動平均）', size=14)
            ax.set_xlabel('日付', size=12)
            ax.set_ylabel('スコア', size=12)
            ax.legend(loc='best')
            ax.grid(True, alpha=0.3)
            fig.autofmt_xdate()

            filepath = f'{self.base_dir}/graphs/{department}_trend.png'
            plt.savefig(filepath, dpi=100, bbox_inches='tight', facecolor='white')
            plt.close()
            graphs['trend'] = filepath

        return graphs

    def analyze_scores(self, df, department):
        """部門ごとの点数分析とグラフ作成"""
        dept_data = df[df['部門'] == department].copy()

        if '日付' in dept_data.columns:
            dept_data['日付'] = pd.to_datetime(dept_data['日付'])
            dept_data = dept_data.sort_values('日付')

        score_columns = [col for col in dept_data.columns
                        if 'スコア' in col or '点数' in col]

        statistics = {}
        for col in score_columns:
            statistics[col] = {
                'mean': dept_data[col].mean(),
                'median': dept_data[col].median(),
                'std': dept_data[col].std(),
                'min': dept_data[col].min(),
                'max': dept_data[col].max(),
                'trend': self._calculate_trend(dept_data[col])
            }

        graphs = self._create_graphs(dept_data, department, score_columns)

        return {
            'department': department,
            'statistics': statistics,
            'graphs': graphs,
            'data': dept_data
        }

    def summarize_free_text(self, texts, department):
        """自由回答のサマリ化"""
        if not texts or all(pd.isna(t) for t in texts):
            return "自由回答データがありません。"

        valid_texts = [str(t) for t in texts if pd.notna(t) and str(t).strip()]

        if not valid_texts:
            return "有効な自由回答データがありません。"

        combined_text = "\n".join(valid_texts[:30])

        prompt = f"""
        以下は{department}の従業員からのフィードバックです。
        重要なポイントを構造化してまとめてください。

        フィードバック:
        {combined_text}

        以下の形式でまとめてください:

        【ポジティブな意見】（上位3つ）
        1.
        2.
        3.

        【改善要望・課題】（上位3つ）
        1.
        2.
        3.

        【全体的な傾向】
        """

        try:
            response = OPENAI_CLIENT.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "あなたは組織分析の専門家です。"},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=800,
                temperature=0.3
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"⚠️ サマリ化エラー: {e}")
            return "APIエラーのため、サマリ化できませんでした。"

    def generate_insights(self, score_analysis, text_summary, department):
        """インサイト生成"""
        stats_text = ""
        for metric, values in score_analysis['statistics'].items():
            stats_text += f"\n• {metric}: 平均{values['mean']:.1f}点, {values['trend']}"

        prompt = f"""
        {department}の分析結果から、経営層向けのインサイトを生成してください。

        【会社方針】
        {self.company_policy}

        【定量分析結果】
        {stats_text}

        【定性分析結果】
        {text_summary}

        以下の構成で、具体的で実行可能なインサイトを提供してください:

        ## 1. エグゼクティブサマリー（2-3文）
        ## 2. 主要な発見事項（3点）
        ## 3. 推奨アクション（優先度順に3つ）
        ## 4. 期待される成果
        """

        try:
            response = OPENAI_CLIENT.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "あなたは経営コンサルタントです。"},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=self.max_tokens,
                temperature=0.5
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"⚠️ インサイト生成エラー: {e}")
            return "APIエラーのため、インサイト生成できませんでした。"

    def create_word_report(self, department, score_analysis, text_summary, insights):
        """Wordレポートの作成"""
        doc = Document()

        title = doc.add_heading(f'{department}', 0)
        title.alignment = WD_ALIGN_PARAGRAPH.CENTER

        subtitle = doc.add_heading('月次分析レポート', 1)
        subtitle.alignment = WD_ALIGN_PARAGRAPH.CENTER

        doc.add_paragraph(f'\n作成日: {datetime.now().strftime("%Y年%m月%d日")}')
        doc.add_page_break()

        # 1. 定量データ分析
        doc.add_heading('1. 定量データ分析', 1)

        # 統計表
        doc.add_heading('1.1 基本統計量', 2)
        table = doc.add_table(rows=1, cols=6)
        table.style = 'Light Grid Accent 1'

        header_cells = table.rows[0].cells
        headers = ['指標', '平均', '中央値', '最小', '最大', 'トレンド']
        for i, header in enumerate(headers):
            header_cells[i].text = header

        for metric, values in score_analysis['statistics'].items():
            row_cells = table.add_row().cells
            row_cells[0].text = metric
            row_cells[1].text = f"{values['mean']:.1f}"
            row_cells[2].text = f"{values['median']:.1f}"
            row_cells[3].text = f"{values['min']:.0f}"
            row_cells[4].text = f"{values['max']:.0f}"
            row_cells[5].text = values['trend']

        doc.add_paragraph()

        # グラフの挿入
        doc.add_heading('1.2 視覚的分析', 2)
        for graph_type, filepath in score_analysis['graphs'].items():
            if os.path.exists(filepath):
                doc.add_paragraph(f'{graph_type.title()}分析:')
                doc.add_picture(filepath, width=Inches(5.5))
                doc.add_paragraph()

        doc.add_page_break()

        # 2. 定性データ分析
        doc.add_heading('2. 定性データ分析', 1)
        doc.add_paragraph(text_summary)

        # 3. インサイトと提言
        doc.add_heading('3. インサイトと提言', 1)
        doc.add_paragraph(insights)

        filepath = f'{self.base_dir}/{department}_レポート_{datetime.now().strftime("%Y%m%d")}.docx'
        doc.save(filepath)

        return filepath

    def process_department(self, df, department):
        """部門ごとの処理"""
        print(f"\n{'='*50}")
        print(f"📊 {department}の処理を開始...")
        print(f"{'='*50}")

        results = {}

        print("1️⃣ 定量データ分析中...")
        score_analysis = self.analyze_scores(df, department)
        results['score_analysis'] = score_analysis

        print("2️⃣ 自由回答分析中...")
        dept_texts = df[df['部門'] == department]['コメント'].tolist() if 'コメント' in df.columns else []
        text_summary = self.summarize_free_text(dept_texts, department)
        results['text_summary'] = text_summary

        print("3️⃣ インサイト生成中...")
        insights = self.generate_insights(score_analysis, text_summary, department)
        results['insights'] = insights

        print("4️⃣ レポート作成中...")
        word_path = self.create_word_report(department, score_analysis, text_summary, insights)
        results['report_path'] = word_path
        print(f"   ✅ レポート: {os.path.basename(word_path)}")

        print(f"\n✅ {department}の処理完了！")

        return results

print("✅ クラス定義完了（全メソッド含む）")

✅ クラス定義完了（全メソッド含む）


## 実行

In [39]:
# システムのインスタンス化
system = ColabDepartmentReportAutomation(
    model=GPT_MODEL,
    max_tokens=MAX_TOKENS,
    company_policy=COMPANY_POLICY
)

# 処理する部門のリスト
departments = df['部門'].unique()
print(f"処理対象部門: {', '.join(departments)}")
print(f"レポート保存先: {system.base_dir}")
print("\n" + "="*60)

# 各部門の処理
all_results = {}
for dept in departments[:2]:  # まず2部門でテスト
    try:
        results = system.process_department(df, dept)
        all_results[dept] = results
    except Exception as e:
        print(f"エラー: {dept} - {e}")
        continue

# 結果サマリー
print("\n" + "="*60)
print("🎉 処理完了！")
print("="*60)

for dept, results in all_results.items():
    print(f"\n部門: {dept}")
    print(f"レポート: {results.get('report_path', 'N/A')}")

if SAVE_TO_DRIVE:
    print(f"\n📁 レポートはGoogle Driveに保存されました:")
    print(f"   場所: {system.base_dir}")

処理対象部門: 人事部, 企画部, 営業部, 経理部, 開発部
レポート保存先: /content/drive/MyDrive/DepartmentReports


📊 人事部の処理を開始...
1️⃣ 定量データ分析中...
2️⃣ 自由回答分析中...
3️⃣ インサイト生成中...
4️⃣ レポート作成中...
   ✅ レポート: 人事部_レポート_20250907.docx

✅ 人事部の処理完了！

📊 企画部の処理を開始...
1️⃣ 定量データ分析中...
2️⃣ 自由回答分析中...
3️⃣ インサイト生成中...
4️⃣ レポート作成中...
   ✅ レポート: 企画部_レポート_20250907.docx

✅ 企画部の処理完了！

🎉 処理完了！

部門: 人事部
レポート: /content/drive/MyDrive/DepartmentReports/人事部_レポート_20250907.docx

部門: 企画部
レポート: /content/drive/MyDrive/DepartmentReports/企画部_レポート_20250907.docx

📁 レポートはGoogle Driveに保存されました:
   場所: /content/drive/MyDrive/DepartmentReports
